In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load and preprocess Wine dataset
data = load_wine()
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to categorical for TensorFlow
from tensorflow.keras.utils import to_categorical
y_train_tf = to_categorical(y_train)
y_test_tf = to_categorical(y_test)

# ------------------- TensorFlow -------------------
import tensorflow as tf

start_tf = time.time()
model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_tf.fit(X_train, y_train_tf, epochs=10, verbose=0)
pred_tf = np.argmax(model_tf.predict(X_test), axis=1)
acc_tf = accuracy_score(y_test, pred_tf)
end_tf = time.time()
time_tf = end_tf - start_tf

# ------------------- PyTorch -------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 16)
        self.fc2 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_torch, y_train_torch), batch_size=16, shuffle=True)

model_torch = Net()
optimizer = torch.optim.Adam(model_torch.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

start_torch = time.time()
for epoch in range(10):
    for xb, yb in train_loader:
        pred = model_torch(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
end_torch = time.time()

with torch.no_grad():
    pred_torch = model_torch(X_test_torch)
    pred_labels = torch.argmax(pred_torch, axis=1).numpy()
acc_torch = accuracy_score(y_test, pred_labels)
time_torch = end_torch - start_torch

# ------------------- Comparison Table -------------------
df = pd.DataFrame({
    "Framework": ["TensorFlow", "PyTorch"],
    "Accuracy": [acc_tf, acc_torch],
    "Training Time (s)": [round(time_tf, 4), round(time_torch, 4)]
})

print("\nBenchmark Comparison Table:")
print(df.to_string(index=False))

# Optional: comment on framework differences
print("\nNote:")
print("- TensorFlow offers higher-level APIs and easier training workflow.")
print("- PyTorch provides more control and debugging ease but requires more setup.")


C:\Users\hameh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step

Benchmark Comparison Table:
 Framework  Accuracy  Training Time (s)
TensorFlow  0.666667             5.6701
   PyTorch  1.000000             0.4091

Note:
- TensorFlow offers higher-level APIs and easier training workflow.
- PyTorch provides more control and debugging ease but requires more setup.
